In [1]:
#Import earth engine
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-tDs2Kwgz3M-WeCgbOeLAnMWcq14IcmHoTca3tkfJl0&tc=8MXrePgxxXzqFk5E5gJLQTk-pTqNJVFsjY2p31AAaHU&cc=I0PrBGGCd-H_z-vtmQ257hsr8OcHfk8FW3eL4zsp-vs

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhXRbmlPIFDLkyprxXru8MhYNpoXo0_6QS0qMSPuLQH4NO8QJrnC-MY

Successfully saved authorization token.


In [2]:
# Load two 5 year Landsat 7 composite
landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

In [3]:
# Compute NDVI the hard way
ndvi1999 = landsat1999.select('B4').subtract(landsat1999.select('B3')).divide(landsat1999.select('B4').add(landsat1999.select('B3')))

# Compute NDVI the easy way
ndvi2008 = landsat2008.normalizedDifference(["B4", 'B3'])

ndviParams = {'palette': ["#d73027", "#f46d43", "#fdae61", "#fee08b", "#d9ef8b", '#a6d96a', '#66bd63', '#1a9850']}

In [4]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[9, 39], zoom_start = 2)

# Add the layer to the map object.
my_map.add_ee_layer(ndvi1999, ndviParams, 'NDVI 1999')
my_map.add_ee_layer(ndvi2008, ndviParams, 'NDVI 2008')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)